In [ ]:
import pandas as pd

In [ ]:
date_cols = ["date"]
holidays = pd.read_csv("../input/store-sales-time-series-forecasting/holidays_events.csv", parse_dates=date_cols)
oil = pd.read_csv("../input/store-sales-time-series-forecasting/oil.csv", parse_dates=date_cols)
stores = pd.read_csv("../input/store-sales-time-series-forecasting/stores.csv")
transactions = pd.read_csv("../input/store-sales-time-series-forecasting/transactions.csv", parse_dates=date_cols)
train = pd.read_csv("../input/store-sales-time-series-forecasting/train.csv", parse_dates=date_cols)
test = pd.read_csv("../input/store-sales-time-series-forecasting/test.csv", parse_dates=date_cols)

In [ ]:
train_h = train.merge(holidays,left_on='date', right_on='date', how='left')
train_h.head()

In [ ]:
train_h_o = train_h.merge(oil,left_on='date', right_on='date', how='left')
train_h_o.head()

In [ ]:
train_h_o_s = train_h_o.merge(stores,left_on='store_nbr', right_on='store_nbr', how='left')
train_h_o_s.head()

In [ ]:
train_h_o_s_t = train_h_o_s.merge(transactions,left_on=["date","store_nbr"], right_on=["date","store_nbr"], how='left')
train_h_o_s_t.head()

In [ ]:
train_h_o_s_t['transferred'].value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

ax = sns.countplot(y ='transferred', data = train_h_o_s_t, hue = "transferred")

for container in ax.containers:
    ax.bar_label(container)
    
# Show the plot
plt.show()

In [ ]:
#Transferred leaves are less, but what date they have shifted is not specified

In [ ]:
train_h_o_s_t['year'] = pd.DatetimeIndex(train_h_o_s_t['date']).year
train_h_o_s_t['month'] = pd.DatetimeIndex(train_h_o_s_t['date']).month
train_h_o_s_t['day'] = pd.DatetimeIndex(train_h_o_s_t['date']).day
train_h_o_s_t['day_name'] = pd.DatetimeIndex(train_h_o_s_t['date']).strftime('%A')
train_h_o_s_t['week_number'] = train_h_o_s_t['date'].dt.strftime('%U')
train_h_o_s_t['day-number'] = train_h_o_s_t['date'].dt.strftime('%w')
train_h_o_s_t["day-number"] = train_h_o_s_t["day-number"].astype(int)
train_h_o_s_t["Weekend"] = train_h_o_s_t['day_name'].isin(['Saturday', 'Sunday'])
train_h_o_s_t.head()

In [ ]:
ax = sns.barplot(x ='Weekend', y = "sales", data = train_h_o_s_t)

#for container in ax.containers:
#    ax.bar_label(container)
#ax.bar_label(ax.containers[0])
ax.bar_label(ax.containers[0], fmt='%.f')
#ax.set_ylabel('%')
# Show the plot
plt.show()

In [ ]:
ax = sns.barplot(x='Weekend', y='sales', estimator=lambda x: len(x) / len(train_h_o_s_t) * 100, data=train_h_o_s_t)
ax.bar_label(ax.containers[0], fmt='%.f%%')
ax.set_ylabel('%')

It clearly shows that on weekends sales are more, comapre to weekdays

In [ ]:
fig, ax1 = plt.subplots(figsize=(16, 5), dpi=100, ncols = 2)

ax2 = sns.barplot(x ='day_name', y = "sales", data = train_h_o_s_t, ax= ax1[0])
ax3 = sns.barplot(x='day_name', y='sales', estimator=lambda x: len(x) / len(train_h_o_s_t) * 100, data=train_h_o_s_t, ax = ax1[1])
ax2.bar_label(ax2.containers[0], fmt='%.f')
ax3.bar_label(ax3.containers[0], fmt='%.f')
ax3.set_ylabel('%')
plt.show()

The bar chart clearly shows that, Saturday and sunday has more sales.  <br>
Compare saturday, sunday has more sales. <br>
Even though records for all the days are same. <br>

In [ ]:
#I feel it is better to remove this column

In [ ]:
fig, ax1 = plt.subplots(figsize=(30, 5), dpi=100, ncols = 3)

ax2 = sns.barplot(x ='year', y = "sales", data = train_h_o_s_t, ax= ax1[0])
ax3 = sns.barplot(x ='month', y = "sales", data = train_h_o_s_t, ax= ax1[1])
ax4 = sns.barplot(x ='day', y = "sales", data = train_h_o_s_t, ax= ax1[2])
ax2.bar_label(ax2.containers[0], fmt='%.f')
ax3.bar_label(ax3.containers[0], fmt='%.f%%')
ax4.bar_label(ax4.containers[0], fmt='%.f')
plt.show()

Clearly shows that every year sales are increasing <br>
<br>
July and December months has highest sales <br>
<br>
1). Start of the month  <br>
2). Mid of the month  <br>
3). End of the month  <br>
Sales are more compare to other days

In [ ]:
train_h_o_s_t.columns

In [ ]:
train_h_o_s_t_1 = train_h_o_s_t.copy()

In [ ]:
train_h_o_s_t["locale"].isnull().sum() *100 / len(train_h_o_s_t)

locale feature will be deleted

In [ ]:
train_h_o_s_t["locale_name"].isnull().sum() *100 / len(train_h_o_s_t)

local_name deature will be deleted

description feature will be deleted, because it's almost same as local_name and 83% entries are missing

In [ ]:
train_h_o_s_t["transferred"].fillna(False, inplace = True)

In [ ]:
train_h_o_s_t_slice = train_h_o_s_t[['store_nbr', 'family', 'sales', 'onpromotion',
                                     'transferred', 'dcoilwtico','city', 'state', 'type_y', 'cluster', 'transactions', 'year', 'month',
                                     'day', 'day_name', 'week_number', 'day-number', 'Weekend']]

In [ ]:
train_h_o_s_t_slice.head()

In [ ]:
train_h_o_s_t_slice["dcoilwtico"].isnull().sum() *100 / len(train_h_o_s_t_slice)

In [ ]:
import numpy as np

In [ ]:
train_h_o_s_t_slice["dcoilwtico"] = train_h_o_s_t_slice["dcoilwtico"].fillna(np.nanmedian(train_h_o_s_t_slice["dcoilwtico"]))

In [ ]:
train_h_o_s_t_slice["dcoilwtico"].isnull().sum() *100 / len(train_h_o_s_t_slice)

In [ ]:
sns.set_style('whitegrid')
sns.distplot(train_h_o_s_t_slice["dcoilwtico"], kde = False, color ='red', bins = 10)

In [ ]:
sns.scatterplot(x='dcoilwtico', y='sales', data=train_h_o_s_t_slice)

In [ ]:
fig, ax = plt.subplots()
sns.scatterplot(x='dcoilwtico', y='sales', data=train_h_o_s_t_slice, ax = ax)
ax.set_ylim(0,40000)
plt.show()

As per scatter, When oil price is between 40 to 60 sales are good, Surpricingly when it is less than 40, sales are decreasing

In [ ]:
#train_h_o_s_t_slice["store_nbr"].value_counts()
fig, ax1 = plt.subplots(figsize=(30, 5), dpi=100)
ax = sns.barplot(x='store_nbr', y='sales', data=train_h_o_s_t_slice, ax=ax1)
ax.bar_label(ax.containers[0], fmt='%.f%%')
ax.set_ylabel('%')

Clearly there is a relation between sales and store_nbr. <br> 3, 44 to 51 store_nbr's has lot of sales compare to other store_nbr.

In [ ]:
train_h_o_s_t_slice["city"].value_counts()

In [ ]:
fig, ax1 = plt.subplots(figsize=(30, 5), dpi=100)
ax = sns.barplot(x='city', y='sales', data=train_h_o_s_t_slice, ax=ax1)
ax.bar_label(ax.containers[0], fmt='%.f%%')
ax.set_ylabel('%')
plt.show()

Quito and Guayaquil has lot sales

In [ ]:
fig, ax1 = plt.subplots(figsize=(30, 5), dpi=100)
ax = sns.barplot(x='state', y='sales', data=train_h_o_s_t_slice, ax=ax1)
ax.bar_label(ax.containers[0], fmt='%.f%%')
ax.set_ylabel('%')
plt.show()

Pichincha state has lot of sales

In [ ]:
fig, ax1 = plt.subplots(figsize=(30, 5), dpi=100)
ax = sns.barplot(x='type_y', y='sales', data=train_h_o_s_t_slice, ax=ax1)
ax.bar_label(ax.containers[0], fmt='%.f')
plt.show()

A type stores has lot of sales

In [ ]:
fig, ax1 = plt.subplots(figsize=(30, 5), dpi=100)
ax = sns.barplot(x='cluster', y='sales', data=train_h_o_s_t_slice, ax=ax1)
ax.bar_label(ax.containers[0], fmt='%.f')
plt.show()

cluster 5 has more sales <br> 8, 11 and 14 stores has significant sales

In [ ]:
train_h_o_s_t_slice["transactions"].isnull().sum() *100 / len(train_h_o_s_t_slice)

In [ ]:
train_h_o_s_t_slice["transactions"].fillna(np.nanmedian(train_h_o_s_t_slice["transactions"]), inplace = True)

In [ ]:
train_h_o_s_t_slice["transactions"].isnull().sum() *100 / len(train_h_o_s_t_slice)

In [ ]:
train_h_o_s_t_slice.isnull().sum()

In [ ]:
train_h_o_s_t_slice.info()

In [ ]:
train_h_o_s_t_slice.family.value_counts()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#fit_transform() will return encoded values 
train_h_o_s_t_slice["family"] = le.fit_transform(train_h_o_s_t_slice["family"])
train_h_o_s_t_slice["family"].value_counts()

In [ ]:
train_h_o_s_t_slice["transferred"] = le.fit_transform(train_h_o_s_t_slice["transferred"])
train_h_o_s_t_slice["transferred"].value_counts()

In [ ]:
train_h_o_s_t_slice["city"] = le.fit_transform(train_h_o_s_t_slice["city"])
train_h_o_s_t_slice["city"].value_counts()

In [ ]:
train_h_o_s_t_slice["state"] = le.fit_transform(train_h_o_s_t_slice["state"])
train_h_o_s_t_slice["state"].value_counts()

In [ ]:
train_h_o_s_t_slice["type_y"] = le.fit_transform(train_h_o_s_t_slice["type_y"])
train_h_o_s_t_slice["type_y"].value_counts()

In [ ]:
train_h_o_s_t_slice["day_name"] = le.fit_transform(train_h_o_s_t_slice["day_name"])
train_h_o_s_t_slice["day_name"].value_counts()

In [ ]:
train_h_o_s_t_slice["week_number"] = le.fit_transform(train_h_o_s_t_slice["week_number"])
train_h_o_s_t_slice["week_number"].value_counts()

In [ ]:
train_h_o_s_t_slice["Weekend"] = le.fit_transform(train_h_o_s_t_slice["Weekend"])
train_h_o_s_t_slice["Weekend"].value_counts()

In [ ]:
train_h_o_s_t_slice.info()

In [ ]:
df_mean = train_h_o_s_t_slice.groupby(["year", "month", "day", "Weekend"]).sales.agg("mean")
df_mean = df_mean.reset_index()
df_mean.columns = ["year", "month", "day", "Weekend", "ymdwend_mean"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_mean,on=["year", "month", "day", "Weekend"], how='left')
train_h_o_s_t_slice.head()

In [ ]:
df_median = train_h_o_s_t_slice.groupby(["year", "month", "day", "Weekend"]).sales.agg("median")
df_median = df_median.reset_index()
df_median.columns = ["year", "month", "day", "Weekend", "ymdwend_median"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_median,on=["year", "month", "day", "Weekend"], how='left')
train_h_o_s_t_slice.head()

In [ ]:
df_std = train_h_o_s_t_slice.groupby(["year", "month", "day", "Weekend"]).sales.agg("std")
df_std = df_std.reset_index()
df_std.columns = ["year", "month", "day", "Weekend", "ymdwend_std"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_std,on=["year", "month", "day", "Weekend"], how='left')
train_h_o_s_t_slice.head()

In [ ]:
df_mean = train_h_o_s_t_slice.groupby(["year", "month", "day", "week_number"]).sales.agg("mean")
df_mean = df_mean.reset_index()
df_mean.columns = ["year", "month", "day", "week_number", "ymdwnum_mean"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_mean,on=["year", "month", "day", "week_number"], how='left')

df_median = train_h_o_s_t_slice.groupby(["year", "month", "day", "week_number"]).sales.agg("median")
df_median = df_median.reset_index()
df_median.columns = ["year", "month", "day", "week_number", "ymdwnum_median"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_median,on=["year", "month", "day", "week_number"], how='left')

df_std = train_h_o_s_t_slice.groupby(["year", "month", "day", "week_number"]).sales.agg("std")
df_std = df_std.reset_index()
df_std.columns = ["year", "month", "day", "week_number", "ymdwnum_std"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_std,on=["year", "month", "day", "week_number"], how='left')
train_h_o_s_t_slice.head()

In [ ]:
df_mean = train_h_o_s_t_slice.groupby(["year", "month", "day", "day_name"]).sales.agg("mean")
df_mean = df_mean.reset_index()
df_mean.columns = ["year", "month", "day", "day_name", "ymdname_mean"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_mean,on=["year", "month", "day", "day_name"], how='left')

df_median = train_h_o_s_t_slice.groupby(["year", "month", "day", "day_name"]).sales.agg("median")
df_median = df_median.reset_index()
df_median.columns = ["year", "month", "day", "day_name", "ymdname_median"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_median,on=["year", "month", "day", "day_name"], how='left')

df_std = train_h_o_s_t_slice.groupby(["year", "month", "day", "day_name"]).sales.agg("std")
df_std = df_std.reset_index()
df_std.columns = ["year", "month", "day", "day_name", "ymdname_std"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_std,on=["year", "month", "day", "day_name"], how='left')
train_h_o_s_t_slice.head()

In [ ]:
df_mean = train_h_o_s_t_slice.groupby(["year", "month", "day", "cluster"]).sales.agg("mean")
df_mean = df_mean.reset_index()
df_mean.columns = ["year", "month", "day", "cluster", "ymdclu_mean"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_mean,on=["year", "month", "day", "cluster"], how='left')

df_median = train_h_o_s_t_slice.groupby(["year", "month", "day", "cluster"]).sales.agg("median")
df_median = df_median.reset_index()
df_median.columns = ["year", "month", "day", "cluster", "ymdclu_median"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_median,on=["year", "month", "day", "cluster"], how='left')

df_std = train_h_o_s_t_slice.groupby(["year", "month", "day", "cluster"]).sales.agg("std")
df_std = df_std.reset_index()
df_std.columns = ["year", "month", "day", "cluster", "ymdclu_std"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_std,on=["year", "month", "day", "cluster"], how='left')
train_h_o_s_t_slice.head()

In [ ]:
df_mean = train_h_o_s_t_slice.groupby(["year", "month", "day", "type_y"]).sales.agg("mean")
df_mean = df_mean.reset_index()
df_mean.columns = ["year", "month", "day", "type_y", "ymdty_mean"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_mean,on=["year", "month", "day", "type_y"], how='left')

df_median = train_h_o_s_t_slice.groupby(["year", "month", "day", "type_y"]).sales.agg("median")
df_median = df_median.reset_index()
df_median.columns = ["year", "month", "day", "type_y", "ymdty_median"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_median,on=["year", "month", "day", "type_y"], how='left')

df_std = train_h_o_s_t_slice.groupby(["year", "month", "day", "type_y"]).sales.agg("std")
df_std = df_std.reset_index()
df_std.columns = ["year", "month", "day", "type_y", "ymdty_std"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_std,on=["year", "month", "day", "type_y"], how='left')
train_h_o_s_t_slice.head()

In [ ]:
df_mean = train_h_o_s_t_slice.groupby(["year", "month", "day", "state"]).sales.agg("mean")
df_mean = df_mean.reset_index()
df_mean.columns = ["year", "month", "day", "state", "ymdstate_mean"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_mean,on=["year", "month", "day", "state"], how='left')

df_median = train_h_o_s_t_slice.groupby(["year", "month", "day", "state"]).sales.agg("median")
df_median = df_median.reset_index()
df_median.columns = ["year", "month", "day", "state", "ymdstate_median"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_median,on=["year", "month", "day", "state"], how='left')

df_std = train_h_o_s_t_slice.groupby(["year", "month", "day", "state"]).sales.agg("std")
df_std = df_std.reset_index()
df_std.columns = ["year", "month", "day", "state", "ymdstate_std"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_std,on=["year", "month", "day", "state"], how='left')
train_h_o_s_t_slice.head()

In [ ]:
df_mean = train_h_o_s_t_slice.groupby(["year", "month", "day", "city"]).sales.agg("mean")
df_mean = df_mean.reset_index()
df_mean.columns = ["year", "month", "day", "city", "ymdcity_mean"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_mean,on=["year", "month", "day", "city"], how='left')

df_median = train_h_o_s_t_slice.groupby(["year", "month", "day", "city"]).sales.agg("median")
df_median = df_median.reset_index()
df_median.columns = ["year", "month", "day", "city", "ymdcity_median"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_median,on=["year", "month", "day", "city"], how='left')

df_std = train_h_o_s_t_slice.groupby(["year", "month", "day", "city"]).sales.agg("std")
df_std = df_std.reset_index()
df_std.columns = ["year", "month", "day", "city", "ymdcity_std"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_std,on=["year", "month", "day", "city"], how='left')
train_h_o_s_t_slice.head()

In [ ]:
train_h_o_s_t_slice["onpromotion"].value_counts()  

In [ ]:
df_mean = train_h_o_s_t_slice.groupby(["year", "month", "day", "onpromotion"]).sales.agg("mean")
df_mean = df_mean.reset_index()
df_mean.columns = ["year", "month", "day", "onpromotion", "ymdprom_mean"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_mean,on=["year", "month", "day", "onpromotion"], how='left')

df_median = train_h_o_s_t_slice.groupby(["year", "month", "day", "onpromotion"]).sales.agg("median")
df_median = df_median.reset_index()
df_median.columns = ["year", "month", "day", "onpromotion", "ymdprom_median"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_median,on=["year", "month", "day", "onpromotion"], how='left')

df_std = train_h_o_s_t_slice.groupby(["year", "month", "day", "onpromotion"]).sales.agg("std")
df_std = df_std.reset_index()
df_std.columns = ["year", "month", "day", "onpromotion", "ymdprom_std"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_std,on=["year", "month", "day", "onpromotion"], how='left')
train_h_o_s_t_slice.head()

In [ ]:
df_mean = train_h_o_s_t_slice.groupby(["year", "month", "day", "family"]).sales.agg("mean")
df_mean = df_mean.reset_index()
df_mean.columns = ["year", "month", "day", "family", "ymdfam_mean"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_mean,on=["year", "month", "day", "family"], how='left')

df_median = train_h_o_s_t_slice.groupby(["year", "month", "day", "family"]).sales.agg("median")
df_median = df_median.reset_index()
df_median.columns = ["year", "month", "day", "family", "ymdfam_median"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_median,on=["year", "month", "day", "family"], how='left')

df_std = train_h_o_s_t_slice.groupby(["year", "month", "day", "family"]).sales.agg("std")
df_std = df_std.reset_index()
df_std.columns = ["year", "month", "day", "family", "ymdfam_std"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_std,on=["year", "month", "day", "family"], how='left')
train_h_o_s_t_slice.head()

In [ ]:
train_h_o_s_t_slice.to_csv("train_host_slice.csv")

In [ ]:
train_h_o_s_t_slice = pd.read_csv("../input/train-imputed/train_host_slice.csv")

In [ ]:
df_mean = train_h_o_s_t_slice.groupby(["year", "month", "day", "store_nbr"]).sales.agg("mean")
df_mean = df_mean.reset_index()
df_mean.columns = ["year", "month", "day", "store_nbr", "ymdstore_mean"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_mean,on=["year", "month", "day", "store_nbr"], how='left')

df_median = train_h_o_s_t_slice.groupby(["year", "month", "day", "store_nbr"]).sales.agg("median")
df_median = df_median.reset_index()
df_median.columns = ["year", "month", "day", "store_nbr", "ymdstore_median"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_median,on=["year", "month", "day", "store_nbr"], how='left')

df_std = train_h_o_s_t_slice.groupby(["year", "month", "day", "store_nbr"]).sales.agg("std")
df_std = df_std.reset_index()
df_std.columns = ["year", "month", "day", "store_nbr", "ymdstore_std"]
train_h_o_s_t_slice = train_h_o_s_t_slice.merge(df_std,on=["year", "month", "day", "store_nbr"], how='left')
train_h_o_s_t_slice.head()

In [ ]:
train_h_o_s_t_slice = train_h_o_s_t_slice.drop(['Unnamed: 0'], axis = 1)
train_h_o_s_t_slice.head()

In [ ]:
train_h_o_s_t_slice.shape

In [ ]:
from sklearn.feature_selection import VarianceThreshold
threshold_n=0.95
sel = VarianceThreshold(threshold=(threshold_n* (1 - threshold_n) ))
sel_var=sel.fit_transform(train_h_o_s_t_slice)
train_h_o_s_t_slice_RVar = train_h_o_s_t_slice[train_h_o_s_t_slice.columns[sel.get_support(indices=True)]] 

In [ ]:
set(train_h_o_s_t_slice.columns)-set(train_h_o_s_t_slice_RVar.columns)

transferred has less than 5% variance, so its is better to remove it

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
dfCorr = train_h_o_s_t_slice_RVar.corr()
filteredDf = dfCorr[((dfCorr >= .9) | (dfCorr <= -.9)) & (dfCorr !=1.000)]
plt.figure(figsize=(30,10))
sns.heatmap(filteredDf, annot=True, cmap="Reds")
plt.show()

looks like we dont have negative corelation

In [ ]:
dfCorr.to_csv("correlation.csv")

month = week_number

ymdwend_mean = ymdwend_std = ymdwnum_mean = ymdwnum_std = ymdname_mean = ymdname_std

ymdwend_median = ymdwnum_median = ymdname_median

ymdclu_mean = ymdclu_std
ymdty_mean = ymdty_std


ymdstate_mean = ymdstate_std = ymdcity_mean = ymdcity_std

ymdstate_median = ymdcity_median
ymdprom_mean = ymdprom_median

ymdfam_mean = ymdfam_median = ymdfam_std

ymdstore_mean = ymdstore_std

These are correlated, I am going to consider the first variable and I will remove other variables.

In [ ]:
train_h_o_s_t_slice_RVar.columns

In [ ]:
col = set(['store_nbr', 'family', 'sales', 'onpromotion', 'dcoilwtico', 'city',
       'state', 'type_y', 'cluster', 'transactions', 'year', 'month', 'day',
       'day_name', 'week_number', 'day-number', 'Weekend', 'ymdwend_mean',
       'ymdwend_median', 'ymdwend_std', 'ymdwnum_mean', 'ymdwnum_median',
       'ymdwnum_std', 'ymdname_mean', 'ymdname_median', 'ymdname_std',
       'ymdclu_mean', 'ymdclu_median', 'ymdclu_std', 'ymdty_mean',
       'ymdty_median', 'ymdty_std', 'ymdstate_mean', 'ymdstate_median',
       'ymdstate_std', 'ymdcity_mean', 'ymdcity_median', 'ymdcity_std',
       'ymdprom_mean', 'ymdprom_median', 'ymdprom_std', 'ymdfam_mean',
       'ymdfam_median', 'ymdfam_std', 'ymdstore_mean', 'ymdstore_median',
       'ymdstore_std']) - set(["week_number", "ymdwend_std", "ymdwnum_mean", "ymdwnum_std", "ymdname_mean", "ymdname_std",
                          "ymdwnum_median", "ymdname_median", "ymdclu_std", "ymdty_std", "ymdstate_std", "ymdcity_mean", "ymdcity_std",
                          "ymdcity_median", "ymdprom_median", "ymdfam_median", "ymdfam_std", "ymdstore_std"])
col = list(col)
col

In [ ]:
col = ['state',
 'dcoilwtico',
 'ymdclu_mean',
 'year',
 'day',
 'ymdty_median',
 'ymdty_mean',
 'store_nbr',
 'ymdprom_std',
 'ymdstate_mean',
 'ymdstate_median',
 'family',
 'cluster',
 'Weekend',
 'ymdwend_median',
 'ymdfam_mean',
 'transactions',
 'ymdclu_median',
 'month',
 'onpromotion',
 'city',
 'ymdwend_mean',
 'day-number',
 'day_name',
 'sales',
 'type_y',
 'ymdprom_mean']

In [ ]:
train_h_o_s_t_slice_RVar_Rcorr = train_h_o_s_t_slice[col]
train_h_o_s_t_slice_RVar_Rcorr.head()

In [ ]:
train_h_o_s_t_slice_RVar_Rcorr.isnull().sum()

In [ ]:
import numpy as np
train_h_o_s_t_slice_RVar_Rcorr["ymdprom_std"].fillna(np.nanmedian(train_h_o_s_t_slice_RVar_Rcorr["ymdprom_std"]), inplace = True)

In [ ]:
y = train_h_o_s_t_slice_RVar_Rcorr["sales"]
X = train_h_o_s_t_slice_RVar_Rcorr.drop(["sales"], axis = 1)

In [ ]:
col1 = X.columns

In [ ]:
from sklearn.preprocessing import StandardScaler
  
scalar = StandardScaler()
  
# fitting
scalar.fit(X)
scaled_data = scalar.transform(X)
type(scaled_data)

In [ ]:
X_df = pd.DataFrame(scaled_data)
X_df.columns = col1
X_df.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.2, random_state=42)

In [ ]:
import statsmodels.api as sm
X_train = sm.add_constant(X_train)
est = sm.OLS(y_train, X_train).fit()
est.summary()

dcoilwtico has 0.286 p value which is less than 95% confidence, will remove that and check the P value

In [ ]:
X_train = X_train.drop(["dcoilwtico"], axis = 1)

In [ ]:
X_train = sm.add_constant(X_train)
est = sm.OLS(y_train, X_train).fit()
est.summary()

ymdstate_median will be removed

In [ ]:
X_train = X_train.drop(["ymdstate_median"], axis = 1)
X_train = sm.add_constant(X_train)
est = sm.OLS(y_train, X_train).fit()
est.summary()

ymdty_median will be removed

In [ ]:
X_train = X_train.drop(["ymdty_median"], axis = 1)
X_train = sm.add_constant(X_train)
est = sm.OLS(y_train, X_train).fit()
est.summary()

day_name will be removed

In [ ]:
X_train = X_train.drop(["day_name"], axis = 1)
X_train = sm.add_constant(X_train)
est = sm.OLS(y_train, X_train).fit()
est.summary()

day will be removed

In [ ]:
X_train = X_train.drop(["day"], axis = 1)
X_train = sm.add_constant(X_train)
est = sm.OLS(y_train, X_train).fit()
est.summary()

In [ ]:
col2 = X_train.columns
col2

In [ ]:
X_test = X_test[['state', 'ymdclu_mean', 'year', 'ymdty_mean', 'store_nbr',
       'ymdprom_std', 'ymdstate_mean', 'family', 'cluster', 'Weekend',
       'ymdwend_median', 'ymdfam_mean', 'transactions', 'ymdclu_median',
       'month', 'onpromotion', 'city', 'ymdwend_mean', 'day-number', 'type_y',
       'ymdprom_mean']]
X_test.head()

In [ ]:
X_train = X_train[['state', 'ymdclu_mean', 'year', 'ymdty_mean', 'store_nbr',
       'ymdprom_std', 'ymdstate_mean', 'family', 'cluster', 'Weekend',
       'ymdwend_median', 'ymdfam_mean', 'transactions', 'ymdclu_median',
       'month', 'onpromotion', 'city', 'ymdwend_mean', 'day-number', 'type_y',
       'ymdprom_mean']]
X_train.head()

In [ ]:
from sklearn.linear_model import LinearRegression
regr = LinearRegression()
  
regr.fit(X_train, y_train)
print(regr.score(X_test, y_test))

In [ ]:
from sklearn.tree import DecisionTreeRegressor 

# create a regressor object
regressor_dt = DecisionTreeRegressor(random_state = 0) 
  
# fit the regressor with X and Y data
regressor_dt.fit(X_train, y_train)
print(regressor_dt.score(X_test, y_test))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
 
 # create regressor object
regressor_rf = RandomForestRegressor(n_estimators = 100, random_state = 0)
 
# fit the regressor with x and y data
regressor_rf.fit(X_train, y_train)
print(regressor_rf.score(X_test, y_test))

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [ ]:
rfr = RandomForestRegressor(random_state=101)
rfecv = RFECV(estimator=rfr, step=100, cv=3, scoring='accuracy', verbose = 2)
rfecv.fit(X, y)

In [ ]:
print('Optimal number of features: {}'.format(rfecv.n_features_))

In [ ]:
dset = pd.DataFrame()
dset['attr'] = X.columns
dset['importance'] = rfecv.estimator_.feature_importances_

dset = dset.sort_values(by='importance', ascending=False)


plt.figure(figsize=(16, 14))
plt.barh(y=dset['attr'], width=dset['importance'], color='#1976D2')
plt.title('RFECV - Feature Importances', fontsize=20, fontweight='bold', pad=20)
plt.xlabel('Importance', fontsize=14, labelpad=20)
plt.show()

In [ ]:
'''from tqdm import tqdm
for i in tqdm(range(len(train_h_o_s_t))):
    if train_h_o_s_t.loc[i,"day-number"] == 5 or train_h_o_s_t.loc[i,"day-number"] == 1:
        train_h_o_s_t.loc[i,"holiday_count"] = 3
    elif train_h_o_s_t.loc[i,"day-number"] == 5 and train_h_o_s_t.loc[i,"day-number"] == 1:
        train_h_o_s_t.loc[i,"holiday_count"] = 4
    elif train_h_o_s_t.loc[i,"day-number"] == 0 or train_h_o_s_t.loc[i,"day-number"] == 6:
        train_h_o_s_t.loc[i,"holiday_count"] = 2
    else:
        train_h_o_s_t.loc[i,"holiday_count"] = 1'''
#train_h_o_s_t.to_csv("train_host.csv")